# Sensoriamento Espectral Inteligente para Coexistência em Mega-Constelações de Satélites LEO
**Autor:** Lana Alves Vieira Gonzaga

**Data:** Novembro de 2025

**Descrição:** Este projeto implementa uma Rede Neural Convolucional (CNN) para classificar 24 tipos de modulação de rádio a partir do dataset RadioML 2018.01A. O objetivo é analisar o desempenho do modelo em diferentes condições de sinal-ruído (SNR). Esse tipo de modelo de classificação é utilizado como base para alocação dinâmica de espectro em sistemas de rádio cognitivo para comunicação eficiente de satélites. Algumas das tecnologias e funcionalidades utilizadas: TensorFlow, Keras, Python, HDF5, pipeline de dados otimizado, classificador CNN e agente cognitivo com detetecção híbrida.

1. Imports e Instalações

In [ ]:
# Instalação de Dependências
# NOTA: Os comandos abaixo são específicos para configuração do ambiente Kaggle/Colab.
# Se estiver a executar localmente, instale as dependências via 'requirements.txt'.

# !pip install kagglehub[pandas-datasets]

# Correção de conflito de versão do Protobuf (Específico para Kaggle)
# !pip uninstall -y protobuf
# !pip install protobuf==3.20.3

In [ ]:
# Imports

import numpy as np
import sys
import h5py
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.patches as mpatches
from IPython.display import FileLink
import seaborn as sns
import json
import warnings
import os
import kagglehub
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from pathlib import Path

warnings.filterwarnings('ignore')

2. Configuração do kaggle para download do dataset "RadioML 2018.01A":

In [ ]:
# 2.1. Configuração da API do Kaggle, adaptada para o ambiente em que for executada.

def setup_kaggle_api():

  !pip install -q kaggle

  IN_COLAB = 'google.colab' in sys.modules

  kaggle_json_path = Path.home() / '.kaggle' / 'kaggle.json'

  if IN_COLAB:
        print("Ambiente Google Colab detectado.")
        if not kaggle_json_path.exists():
            print("Faça o upload do ficheiro kaggle.json.")
            from google.colab import files
            uploaded = files.upload()

            if 'kaggle.json' in uploaded:
                print("kaggle.json recebido. Configurando.")

                kaggle_dir = Path.home() / '.kaggle'
                kaggle_dir.mkdir(exist_ok=True, parents=True)

                with open(kaggle_json_path, 'wb') as f:
                    f.write(uploaded['kaggle.json'])
            else:
                print("Upload cancelado ou ficheiro incorreto.")
                return False

  # Fora do colab.
  else:
        print("Ambiente local ou desconhecido detectado.")

        if not kaggle_json_path.exists():
            print(f"ERRO: Ficheiro kaggle.json não encontrado em '{kaggle_json_path}'.")
            print("Por favor, descarregue o seu token da API do Kaggle e coloque-o nesse local.")
            return False

  !chmod 600 {kaggle_json_path}

setup_kaggle_api()

In [ ]:
# 2.2. Download do dataset "RadioML 2018.01A".

try:
    path = kagglehub.dataset_download("pinxau1000/radioml2018")
    print(f"\nDownload concluído. Os ficheiros estão em: '{path}'")

    print("\nArquivos encontrados no diretório do dataset:")
    found_files = []
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            file_path = os.path.join(dirname, filename)
            if file_path.endswith(('.h5', '.hdf5')):
                found_files.append(file_path)
                print(f"  • {file_path}")

    if found_files:
        HDF5_FILE_PATH = found_files[0]
        print(f"\nCaminho do ficheiro HDF5 para carregar: '{HDF5_FILE_PATH}'")
    else:
        print("\nNenhum ficheiro HDF5 encontrado no diretório descarregado.")

except Exception as e:
      print(f"\nOcorreu um erro durante o download: {e}")

In [ ]:
# 2.3. Passar arquivo para o disco local

import os
import shutil
import time

print("COPIANDO ARQUIVO PARA DISCO LOCAL")

print(f"\n Arquivo atual:")
print(f"   Caminho: {HDF5_FILE_PATH}")
print(f"   Tamanho: {os.path.getsize(HDF5_FILE_PATH) / 1e9:.2f} GB")


local_path = '/content/radioml_local.hdf5'


if os.path.exists(local_path):
    print(f"\n Arquivo local já existe: {local_path}")
    print(f"   Tamanho: {os.path.getsize(local_path) / 1e9:.2f} GB")
else:
    print(f"\n Copiando para: {local_path}")
    print("   Isso vai levar alguns minutos.")

    start = time.time()
    shutil.copy2(HDF5_FILE_PATH, local_path)
    elapsed = time.time() - start

    print(f"   Cópia concluída em {elapsed/60:.1f} minutos.")
    print(f"   Tamanho: {os.path.getsize(local_path) / 1e9:.2f} GB")


HDF5_FILE_PATH = local_path
print(f"\n Novo caminho: {HDF5_FILE_PATH}")

3. Pré-processamento dos dados:

In [ ]:
# 3.1. Divisão em índices das informações do dataset, para não sobrecarregar a RAM com o conjunto inteiro de dados de uma vez só.

HDF5_FILE_PATH = '/content/radioml_local.hdf5'

with h5py.File(HDF5_FILE_PATH, 'r') as f:
    total_samples = f['X'].shape[0]
    num_classes = f['Y'].shape[1]
print(f"Total de amostras no dataset: {total_samples}")
print(f"Número de classes de modulação: {num_classes}")

all_indices = np.arange(total_samples)

with h5py.File(HDF5_FILE_PATH, 'r') as f:
    labels_for_stratify = f['Y'][:]

# Primeiro split: 80% treino+validação / 20% teste.
indices_temp, test_indices = train_test_split(
    all_indices, test_size=0.2, random_state=42, stratify=labels_for_stratify
)

labels_temp = labels_for_stratify[indices_temp]

# Segundo split: divide treino+validação em 80% treino / 20% validação.
train_indices, val_indices = train_test_split(
    indices_temp, test_size=0.1 / (1 - 0.2), random_state=42, stratify=labels_temp
)

print(f"\nNúmero de índices de treino: {len(train_indices)}")
print(f"Número de índices de validação: {len(val_indices)}")
print(f"Número de índices de teste: {len(test_indices)}")

Para otimizar o tempo de treinamento em ambiente de nuvem (Kaggle), optou-se por carregar um subconjunto estratificado de 1 milhão de amostras diretamente na RAM, utilizando precisão float16 para eficiência de memória.

In [ ]:
# 3.2. Carregamento pra RAM e configuração do modelo CNN

# 1. CONFIGURAÇÕES
# (Confirme se este caminho está certo para o seu ambiente.)
# Se usou o kagglehub antes, pode usar a variável 'path' ou 'HDF5_FILE_PATH' que ele gerou.

NUM_SAMPLES_TO_LOAD = 1000000 

print(f" Carregando {NUM_SAMPLES_TO_LOAD} amostras para a RAM.")

with h5py.File(HDF5_FILE_PATH, 'r') as f:
    X = f['X'][:NUM_SAMPLES_TO_LOAD]

    print("Normalizando dados.")

    max_val = np.max(np.abs(X)) 
    X = X / max_val
    
    X = X.astype(np.float16)
    
    Y = f['Y'][:NUM_SAMPLES_TO_LOAD].astype(np.float16)

print(f"Dados carregados. X shape: {X.shape}")

y_indices = np.argmax(Y, axis=1)

print("Dividindo em Treino e Validação.")
X_train, X_val, Y_train, Y_val = train_test_split(
    X, Y, 
    test_size=0.3,
    random_state=42, 
    stratify=y_indices
)

del X, Y, y_indices
gc.collect()

print(f"Treino: {X_train.shape[0]} amostras | Validação: {X_val.shape[0]} amostras")

def create_model():
    model = keras.models.Sequential([
        layers.Input(shape=(1024, 2)),
        
        layers.Conv1D(64, 8, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(2),
        
        layers.Conv1D(128, 8, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(2),
        
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5), 
        layers.Dense(24, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = create_model()
model.summary()

batch_size = 1024

my_callbacks = [
    callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001, verbose=1),
    callbacks.ModelCheckpoint('melhor_modelo_radio.keras', monitor='val_accuracy', save_best_only=True, verbose=1)
]

In [ ]:
# 3.3. Visuzalização dos dados

def visualize_ram_data(X, Y, num_samples=4, figsize=(16, 10)):
    indices = np.random.choice(len(X), num_samples, replace=False)
    
    modulation_names = ['8PSK', 'AM-DSB', 'AM-SSB', 'BPSK', 'CPFSK', 'FM', 'GFSK', 
                        'PAM4', 'QAM16', 'QAM64', 'QPSK', 'WBFM', '16APSK', '32APSK',
                        '64APSK', '128APSK', '16QAM', '32QAM', '64QAM', '128QAM',
                        '256QAM', 'AM-DSB-SC', 'AM-SSB-SC', 'OQPSK']

    fig = plt.figure(figsize=figsize)
    gs = GridSpec(num_samples, 3, figure=fig, hspace=0.4, wspace=0.3)

    for i, idx in enumerate(indices):
        signal = X[idx]
        label_onehot = Y[idx]
        
        mod_name = modulation_names[np.argmax(label_onehot)]

        I, Q = signal[:, 0], signal[:, 1]
        time_steps = np.arange(len(I))

        ax1 = fig.add_subplot(gs[i, 0])
        ax1.plot(time_steps, I, 'b-', alpha=0.7, label='I')
        ax1.plot(time_steps, Q, 'r-', alpha=0.7, label='Q')
        ax1.set_title(f'{mod_name} - Tempo')
        ax1.legend(loc='upper right', fontsize='small')
        ax1.grid(True, alpha=0.3)

        ax2 = fig.add_subplot(gs[i, 1])
        ax2.scatter(I, Q, c=time_steps, cmap='viridis', s=5, alpha=0.6)
        ax2.set_title(f'{mod_name} - Constelação')
        ax2.axis('equal')
        ax2.grid(True, alpha=0.3)

        ax3 = fig.add_subplot(gs[i, 2])
        signal_complex = I + 1j * Q
        fft = np.fft.fftshift(np.fft.fft(signal_complex))
        power = 10 * np.log10(np.abs(fft)**2 + 1e-10)
        freqs = np.fft.fftshift(np.fft.fftfreq(len(signal_complex)))
        ax3.plot(freqs, power, 'g-')
        ax3.set_title(f'{mod_name} - Espectro')
        ax3.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('amostras_sinais_ram.png', dpi=150)
    plt.show()

visualize_ram_data(X_train, Y_train)


4. Treinamento do Modelo

In [ ]:
# 4.1. Treinamento

print("Iniciando o treinamento.")

BATCH_SIZE = 1024

history = model.fit(
    x=X_train,
    y=Y_train,
    validation_data=(X_val, Y_val), 
    batch_size=1024,
    epochs=15,
    callbacks=my_callbacks,
    verbose=1
)

print("Treinamento concluído.")

import matplotlib.pyplot as plt

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)

    plt.figure(figsize=(14, 5))

    # Gráfico de Acurácia
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'bo-', label='Treino')
    plt.plot(epochs, val_acc, 'ro-', label='Validação')
    plt.title('Acurácia de Treino e Validação')
    plt.xlabel('Épocas')
    plt.ylabel('Acurácia')
    plt.legend()
    plt.grid(True)

    # Gráfico de Perda
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'bo-', label='Treino')
    plt.plot(epochs, val_loss, 'ro-', label='Validação')
    plt.title('Perda (Loss) de Treino e Validação')
    plt.xlabel('Épocas')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    nome_arquivo = f'graficos_treino.png'
    plt.savefig(nome_arquivo, dpi=300, bbox_inches='tight')
    print(f"Imagem salva como: {nome_arquivo}")

    plt.savefig('figura_sinais.pdf', bbox_inches='tight')

    plt.show()

plot_history(history)

from IPython.display import FileLink

FileLink(r'melhor_modelo_radio.keras')

5. Verificações

In [ ]:
# 5.1. Gráfico de Acurácia vs SNR

with h5py.File(HDF5_FILE_PATH, 'r') as f:
    Y_full = f['Y'][:NUM_SAMPLES_TO_LOAD]
    Z_full = f['Z'][:NUM_SAMPLES_TO_LOAD]

y_indices_full = np.argmax(Y_full, axis=1)

_, Z_test_aligned, _, _ = train_test_split(
    Z_full, Y_full, 
    test_size=0.3, 
    random_state=42, 
    stratify=y_indices_full
)

print("Gerando previsões.")
y_pred_prob = model.predict(X_val, verbose=0)
y_pred_classes = np.argmax(y_pred_prob, axis=1)
y_true_classes = np.argmax(Y_val, axis=1)

snrs = sorted(list(np.unique(Z_test_aligned)))
acc_by_snr = []

for snr in snrs:
    if Z_test_aligned.ndim > 1:
        mask = (Z_test_aligned == snr)[:, 0]
    else:
        mask = (Z_test_aligned == snr)
        
    if np.sum(mask) > 0:
        acc = np.mean(y_pred_classes[mask] == y_true_classes[mask])
        acc_by_snr.append(acc)
    else:
        acc_by_snr.append(0)

plt.figure(figsize=(10, 6))
plt.plot(snrs, acc_by_snr, 'bo-', linewidth=2, label='Sua CNN (RAM)')
plt.axhline(y=1/24, color='r', linestyle='--', label='Aleatório (4%)')
plt.title(f'Acurácia vs. SNR (Média Global: {np.mean(y_pred_classes == y_true_classes):.2%})')
plt.xlabel('SNR (dB)')
plt.ylabel('Acurácia')
plt.grid(True)
plt.legend()
plt.savefig('acuracia_snr.png', dpi=300)
plt.show()

In [ ]:
# 5.2. Matriz de confusão

print("Gerando matriz de confusão.")

modulation_names = [
    'OOK', '4ASK', '8ASK', 'BPSK', 'QPSK', '8PSK', '16PSK', '32PSK',
    '16APSK', '32APSK', '64APSK', '128APSK', '16QAM', '32QAM', '64QAM',
    '128QAM', '256QAM', 'AM-SSB-WC', 'AM-SSB-SC', 'AM-DSB-WC', 'AM-DSB-SC',
    'FM', 'GMSK', 'OQPSK'
]

cm = confusion_matrix(y_true_classes, y_pred_classes)

classes_presentes_indices = np.where(cm.sum(axis=1) > 0)[0]

cm_filtered = cm[np.ix_(classes_presentes_indices, classes_presentes_indices)]

cm_norm = cm_filtered.astype('float') / cm_filtered.sum(axis=1)[:, np.newaxis]

my_modulations = [modulation_names[i] for i in classes_presentes_indices]


plt.figure(figsize=(12, 10))
sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='viridis',
            xticklabels=my_modulations,
            yticklabels=my_modulations)

plt.title('Matriz de Confusão (Classes Treinadas)', fontsize=16)
plt.ylabel('Rótulo Verdadeiro', fontsize=12)
plt.xlabel('Rótulo Previsto', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)

plt.savefig('matriz_confusao_final.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# 5.3. Simulação de canais livres (ruído gaussiano) - Análise de confiança

print("Análise de Confiança: Ruído vs. Sinais Reais")

noise_batch = np.random.normal(0, 1, size=(1000, 1024, 2))
max_val = np.max(np.abs(noise_batch), axis=(1, 2), keepdims=True)
noise_batch = noise_batch / (max_val + 1e-7)

x_real_batch = X_val[:1000]

print("Prevendo ruído.")
pred_noise = model.predict(noise_batch, verbose=0)
print("Prevendo sinais reais.")
pred_signal = model.predict(x_real_batch, verbose=0)

conf_noise = np.max(pred_noise, axis=1)
conf_signal = np.max(pred_signal, axis=1)

plt.figure(figsize=(10, 6))
plt.hist(conf_noise, bins=50, alpha=0.7, label='Ruído (Canal Livre)', color='red')
plt.hist(conf_signal, bins=50, alpha=0.7, label='Sinais Reais (Ocupado)', color='blue')
plt.title('Por que precisamos de Energia? (IA tem excesso de confiança no ruído)')
plt.xlabel('Confiança do Modelo (0 a 1)')
plt.ylabel('Contagem')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Confiança média no Ruído: {np.mean(conf_noise):.4f}")
print("Conclusão: A IA confunde ruído normalizado com sinal. O Agente precisará de um filtro de Energia.")

6. Implementação de Modelo Heurístico Baseado em Regras (Rule-Based Heuristic), para alocação de espectro.

In [ ]:
# 6.1. Funções auxiliares e configuração

CONFIDENCE_THRESHOLD = 0.5 

def generate_noise(shape=(1024, 2)):
    return np.random.normal(0, 0.005, size=shape)

if 'modulation_names' not in locals():
    modulation_names = ['8PSK', 'AM-DSB', 'AM-SSB', 'BPSK', 'CPFSK', 'FM', 'GFSK', 
                        'PAM4', 'QAM16', 'QAM64', 'QPSK', 'WBFM', '16APSK', '32APSK',
                        '64APSK', '128APSK', '16QAM', '32QAM', '64QAM', '128QAM',
                        '256QAM', 'AM-DSB-SC', 'AM-SSB-SC', 'OQPSK']

In [ ]:
# 6.2. Classe do ambiente

import numpy as np

class SpectrumEnvironment:
    def __init__(self, num_channels, X_source, Y_source, mod_names):
        self.num_channels = num_channels
        self.X_source = X_source 
        self.Y_source = Y_source 
        self.mod_names = mod_names
        self.ground_truth = []
        
    def update(self):
        current_signals = []
        self.ground_truth = []
        
        occupancy_rate = 0.6 

        for i in range(self.num_channels):
            if np.random.random() < occupancy_rate:
 
                rand_idx = np.random.randint(0, len(self.X_source))
                
                signal = self.X_source[rand_idx]
                label_onehot = self.Y_source[rand_idx]
                
                label_idx = np.argmax(label_onehot)
                real_label = self.mod_names[label_idx]
                
                current_signals.append(signal)
                self.ground_truth.append(real_label)
            else:
                noise = np.random.normal(0, 0.01, size=(1024, 2))
                current_signals.append(noise)
                self.ground_truth.append("Livre")
                
        return np.array(current_signals)

    def step(self, action_channel):
        actual_state = self.ground_truth[action_channel]
        if actual_state == "Livre":
            return 10, "Sucesso."
        else:
            return -50, f" Colisão ({actual_state})"

In [ ]:
# 6.3. Classe do agente

class CognitiveAgent:
    def __init__(self, model, mod_names):
        self.model = model
        self.mod_names = mod_names
        self.energy_threshold = 0.01 
        
    def sense(self, signals):
        sensed_results = []
        
        predictions = self.model.predict(signals, verbose=0)
        
        for i, signal in enumerate(signals):
            energy = np.mean(np.abs(signal))
            
            if energy < self.energy_threshold:
                sensed_results.append("Livre")
                continue
    
            pred = predictions[i]
            predicted_idx = np.argmax(pred)
            predicted_label = self.mod_names[predicted_idx]
            
            sensed_results.append(predicted_label)
        
        return sensed_results

    def decide(self, sensed_results):
        free_channels = [i for i, status in enumerate(sensed_results) if status == "Livre"]
        
        if not free_channels:
            return None 
        
        return np.random.choice(free_channels)

In [ ]:
# 6.4. Execução

NUM_CHANNELS = 5
STEPS = 50 

env = SpectrumEnvironment(NUM_CHANNELS, X_val, Y_val, modulation_names)
agent = CognitiveAgent(model, modulation_names)

history_actions = []
history_results = []

print(f"Rodando simulação de {STEPS} passos para gerar gráfico.")

for t in range(STEPS):
    signals = env.update()
    
    sensed = agent.sense(signals)
    action = agent.decide(sensed)
    
    if action is not None:
        reward, msg = env.step(action)
        result_type = "Sucesso" if reward > 0 else "Colisão"
        history_actions.append((t, action))
        history_results.append(result_type)
    else:
        history_actions.append((t, -1)) 
        history_results.append("Espera")

plt.figure(figsize=(15, 6))

colors = {"Sucesso": "green", "Colisão": "red", "Espera": "gray"}
markers = {"Sucesso": "o", "Colisão": "X", "Espera": "s"}

for i, (step, channel) in enumerate(history_actions):
    res = history_results[i]
    
    y_pos = channel if channel != -1 else -0.8
    
    plt.scatter(step, y_pos, color=colors[res], marker=markers[res], s=100, zorder=3)
    
    if channel != -1:
        plt.vlines(step, -0.8, channel, color='gray', linestyle=':', alpha=0.3)

plt.title('Timeline da Simulação: Alocação Dinâmica de Espectro', fontsize=16)
plt.xlabel('Tempo (Passos da Simulação)', fontsize=12)
plt.ylabel('Canal de Frequência', fontsize=12)

plt.yticks([-0.8, 0, 1, 2, 3, 4], ["Fila de\nEspera", "Ch 0", "Ch 1", "Ch 2", "Ch 3", "Ch 4"])
plt.ylim(-1.5, 4.5)
plt.grid(True, axis='x', alpha=0.3)

legend_patches = [
    mpatches.Patch(color='green', label='Transmissão Sucesso (Canal Livre)'),
    mpatches.Patch(color='red', label='Colisão (Interferência)'),
    mpatches.Patch(color='gray', label='Agente Aguardou (Sem canais)')
]
plt.legend(handles=legend_patches, loc='upper right')

nome_arquivo = 'resultado_simulacao.png'
plt.savefig(nome_arquivo, dpi=300, bbox_inches='tight')
print(f"Gráfico salvo como: {nome_arquivo}")

plt.show()